In [1]:
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import subprocess
import pybedtools

In [2]:
DATA = Path("../data")
METHYL = DATA / "methylation_peaks"
METHYL_CONCAT = DATA / "methylation_peaks_concat"
ANNO = Path("/export/home/public/agletdinov_shared/annotations")
PREPROCESSING = Path("../preprocessing")
METHYL_INDEX = PREPROCESSING / "methyl_index"
METHYL_BED = PREPROCESSING / "methyl_bed"

In [3]:
import gzip
import pandas as pd

# Функция для чтения GTF-файла и подсчета уникальных генов
def count_genes_in_gtf(gtf_file):
    # Открываем файл
    with gzip.open(gtf_file, 'rt') as f:
        # Считываем строки, относящиеся к генам
        gene_lines = [line for line in f if 'gene' in line and line.split('\t')[2] == 'gene']
        
    # Выделяем ID генов из строк
    gene_ids = set()
    for line in gene_lines:
        # Разбиваем строку на колонки
        columns = line.split('\t')
        # Последняя колонка содержит атрибуты, ищем gene_id
        attributes = columns[8]
        for attribute in attributes.split(';'):
            if 'gene_id' in attribute:
                gene_id = attribute.split('"')[1]
                gene_ids.add(gene_id)
                break
    
    return len(gene_ids)

In [4]:
def get_all_correlated_peaks(hm, to_hdf5, to_save):
    if to_save.exists():
        print(f"Skipped: {to_hdf5.stem}")
        return None
    all_results = []

    # Открываем HDF5 файл
    with h5py.File(to_hdf5, 'r') as f:
        lncRNAs = list(f['lncRNAs_names'][:])
        corrs_matrix = f['corrs_matrix'][:]

    # Читаем пики
    peaks = pd.read_csv(Path("/data/mazurovev/all_marks") / hm / "merged_peaks_first_in_biosample.bed", sep="\t", header=None)

    for i, lncRNA in enumerate(lncRNAs):
        corrs = corrs_matrix[i, :]
        nonzero_indices = np.nonzero(corrs)[0]
        nonzero_corrs = corrs[nonzero_indices]
        nonzero_peaks = ["peak_" + str(j) for j in nonzero_indices]

        # Извлекаем соответствующие пики
        res = peaks[peaks[3].isin(nonzero_peaks)].copy()
        res["corr"] = nonzero_corrs
        res["lnc_ens"] = lncRNA.decode('utf-8')  # Декодируем байтовую строку в строку

        all_results.append(res)

    # Объединяем все результаты в один DataFrame
    final_result = pd.concat(all_results, ignore_index=True)
    
    final_result.to_csv(to_save, sep="\t", compression="gzip", index=False)

In [18]:
def indexing_methyl(chunk, lnc_ens, corr_sign, subdf):
    to_index = METHYL_INDEX
    to_index.mkdir(exist_ok=True, parents=True)
    to_save = to_index / f"methyl-{lnc_ens}-{corr_sign}-.tsv.gz"
    if to_save.exists():
        print(f"Skipped: methyl {lnc_ens} {corr_sign}")
        return None
    subdf.to_csv(to_save, sep="\t", compression="gzip", index=False)

In [19]:
for to_df in METHYL.glob("*.tsv.gz"):
    print(to_df.stem)
    chunk = to_df.stem.split("_")[6]
    df = pd.read_csv(to_df, sep="\t")
    df.rename(columns={"0":"chrom", "1":"start", "2":"end", "3":"name"}, inplace=True)
    df["corr"] = df["corr"].apply(lambda x: np.round(x, 3))
    df["corr_sign"] = np.where(df["corr"] > 0, "plus", "minus")
    for (lnc_ens, corr_sign), subdf in df.groupby(by=["lnc_ens", "corr_sign"]):
        indexing_methyl(chunk=chunk, lnc_ens=lnc_ens, corr_sign=corr_sign, subdf=subdf)

lncRNA_Peaks_Correlations_corrected_non_zero_0.tsv



KeyboardInterrupt



In [6]:
to_csv= METHYL_CONCAT / "all_methil_merged.tsv.gz"
all_methil_merged = pd.read_csv(to_csv, sep="\t", compression="gzip")

In [7]:
len(all_methil_merged)

277825397

In [16]:
all_methil_merged.sort_values(by=["0","1","2"], ascending=False)

,0,1,2,3,4,5,6
23821946,chrY,56884830,56884831,peak_53936927,0.637,ENSG00000256576,plus
41145915,chrY,56884830,56884831,peak_53936927,0.622,ENSG00000280234,plus
129168025,chrY,56884830,56884831,peak_53936927,0.640,ENSG00000272750,plus
41145914,chrY,56884829,56884830,peak_53936926,0.595,ENSG00000280234,plus
121784693,chrY,56884816,56884817,peak_53936925,-0.772,ENSG00000227627,minus
...,...,...,...,...,...,...,...
215251195,chr1,10578,10579,peak_17,-0.655,ENSG00000259495,minus
248981362,chr1,10578,10579,peak_17,-0.624,ENSG00000278989,minus
112933774,chr1,10576,10577,peak_16,-0.642,ENSG00000227888,minus
113001538,chr1,10492,10493,peak_8,-0.653,ENSG00000242759,minus


In [8]:
all_methil_merged_for_bed = all_methil_merged.drop_duplicates(subset=["0","1","2"])[["0","1","2"]]
all_methil_merged_for_bed.shape

(25177317, 3)

In [17]:
all_methil_merged_for_bed_sort = all_methil_merged_for_bed.sort_values(by=["0","1","2"], ascending=False)

In [ ]:
all_methil_merged_for_bed_sort[all_methil_merged_for_bed_sort["1"] == 10541]

In [9]:
bed_concat = pybedtools.BedTool(METHYL_BED / "methyl_concat" /'methyl_concat.bed')

In [12]:
bed_concat_df = pd.read_table(bed_concat.fn, names=['chrom', 'start', 'stop'])

In [13]:
bed_concat_df

,chrom,start,stop
0,chr1,10492,10493
1,chr1,10541,10542
2,chr1,10570,10571
3,chr1,10576,10577
4,chr1,10578,10579
...,...,...,...
17758256,chrY,56884088,56884090
17758257,chrY,56884121,56884122
17758258,chrY,56884235,56884236
17758259,chrY,56884815,56884817


In [ ]:
def tsv_to_bed(to_tsv):
    to_save = METHYL_BED / f"{to_tsv.name.split('.')[0]}.bed"
    if to_save.exists():
        print(f"Skipped: {to_save}")
        return None
    to_save.parent.mkdir(exist_ok=True, parents=True)
    df = pd.read_csv(to_tsv, sep="\t")
    stream = open(to_save, 'w')
    for i in range(len(df)):
        stream.write(f"{df.loc[i, 'chrom']}\t{df.loc[i, 'start']}\t{df.loc[i, 'end']}\n")
    
    stream.close()
    #cmd = f"gzip {to_save}" 
    #subprocess.run(cmd, shell=True, check=True)

In [ ]:
with Parallel(n_jobs=-1) as pool:
    chunks = pool(
        delayed(tsv_to_bed)(to_tsv)
        for to_tsv in METHYL_INDEX.glob("*")
    )